<a href="https://colab.research.google.com/github/tpessoa-dev/plataformas-cognitivas/blob/main/Plataformas_Azure_Treino_Fora_Azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
!git clone https://github.com/elthonf/azure-ml.git

fatal: destination path 'azure-ml' already exists and is not an empty directory.


In [ ]:
!pip install azureml-core
!pip install scikit-learn==0.20.3

In [ ]:
!pip install azureml-dataprep
!pip install azureml-dataset-runtime

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
import sklearn
import math
from azureml.core import Experiment
import random
import pandas as pd
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import RandomForestRegressor as rfr
import uuid
# O azureml-core da versão 1.0.72 ou superior é requerido
# é necessário azureml-dataprep[pandas] na versão 1.1.34 ou superior
from azureml.core import Workspace, Dataset

In [ ]:
from azureml.core import Model

In [ ]:
def get_a_funnyName():
    f = open("azure-ml/datasets/names/funnynames.txt", "r")
    nomes = f.readlines()
    return random.choice(nomes).rstrip('\n')

# Conexão

In [ ]:
#Trocar os códigos abaixo pelos da sua instância!
subscription_id = '5298dbd8-b7e6-432a-b8fc-828328490c29'
resource_group = 'Santander'
workspace_name = 'AreaParaOSantander'

workspace = Workspace(subscription_id, resource_group, workspace_name)

# Carga de dados

In [ ]:
mydf = pd.read_csv('azure-ml/datasets/statistical/BaseDefault01.csv')
mydf.head()

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade,default
0,"Simon, Rodriguez",4472.190323,42.036031,0,0,1,0,0,3,0
1,"Daniel, Castro",4592.774312,48.230662,1,0,1,0,1,2,0
2,"Myhue, Lin",2486.538807,56.881709,0,1,0,0,0,0,1
3,"Destiny, Richardson-Pacheco",2852.340117,51.684021,1,1,0,0,0,2,1
4,"Brittany, Cohen-Wilson",4703.782812,50.729078,1,1,1,0,1,2,0


In [ ]:
dataset = Dataset.get_by_name(workspace, name='Risco-De-Credito')
df = dataset.to_pandas_dataframe()
df.head()

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade,default
0,"Simon, Rodriguez",4472.190323,42.036031,0,0,1,0,0,3,0
1,"Daniel, Castro",4592.774312,48.230662,1,0,1,0,1,2,0
2,"Myhue, Lin",2486.538807,56.881709,0,1,0,0,0,0,1
3,"Destiny, Richardson-Pacheco",2852.340117,51.684021,1,1,0,0,0,2,1
4,"Brittany, Cohen-Wilson",4703.782812,50.729078,1,1,1,0,1,2,0


# Treinamento

Identifica no dataset as variáveis independentes e a variavel alvo

In [ ]:
targetcol = 'default'
y = mydf[targetcol]

Cria experimento

In [ ]:
experiment = Experiment(workspace=workspace, name="Experimento_Banco")

## Treina modelo 01 (Classificador)

Cria uma execução para o experimento

In [ ]:
run = experiment.start_logging(run_id= "Classificador1_" + str( uuid.uuid1() ) )
run.log("Tipo", "Classificador")

In [ ]:
independentcols = ['renda', 'idade', 'etnia', 'sexo', 'casapropria', 'outrasrendas', 'estadocivil', 'escolaridade']
x = mydf[independentcols]
clf = rfc()
clf.fit(X=x, y=y)
clf.independentcols = independentcols

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Avaliações e Logs

In [ ]:
clf_acuracia = clf.score(X=x, y=y)
print("Modelo 01 (classificador), criado com acurácia de: [{0}]".format(clf_acuracia))
run.log("acuracia", clf_acuracia)

Modelo 01 (classificador), criado com acurácia de: [0.98707]


In [ ]:
#Demais logs (não serão comparados)
run.log("Versao sklearn", sklearn.__version__)
run.log("criterion", clf.criterion)
run.log("n_estimators", clf.n_estimators)
run.log("min_samples_leaf", clf.min_samples_leaf)
run.log("max_depth",  str(clf.max_depth))
run.log_list("Inputs", independentcols)

Exporta o modelo

In [ ]:
model_name = "nome_arquivo.pkl"
filename = "outputs/" + model_name
joblib.dump(value=clf, filename=filename)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

['outputs/nome_arquivo.pkl']

Upload do arquivo, deve demorar um pouco

In [ ]:
run.upload_file(name=model_name, path_or_stream=filename)

In [ ]:
ls -l outputs

total 15852
-rw-r--r-- 1 root root 16228543 Mar 16 23:19 nome_arquivo.pkl


Finaliza o experimento

In [ ]:
run.complete()

# Atividade 1 : Análise na Azure

Crie n modelos, e envie para a Azure. Em seguida, compare os resultados dentro da plataforma.  

Obs.:  
n = 8 $\times$ 4 $\times$ 3

In [ ]:
experiment = Experiment(workspace=workspace, name="Experimento_RiscoCredito")

In [ ]:
n_estimators = [1, 2, 3, 5, 10, 20, 30, 100]
max_depth = [1, 2, 3, None]
min_samples_leaf = [10, 5, 1]

# Atividade 2 : Regressor

Treinar um segundo modelo (Regressor sem a variavel etnia)

In [ ]:
independentcols = ['renda', 'idade', 'sexo', 'casapropria', 'outrasrendas', 'estadocivil', 'escolaridade']

- Usar o classificador acima como modelo
- Gravar log de algumas variáveis
- Gravar log da acurácia
- Exportar o modelo em arquivo
- Enviar para AZURE Cloud

In [ ]:
# Upload direto
# modelo = Model.register(workspace=workspace, model_name="ModeloUploadDireto", 
#                        model_framework=Model.Framework.SCIKITLEARN,
#                        model_framework_version='0.20.3',
#                        model_path=filename)